<a href="https://colab.research.google.com/github/jung-youjin/dip-pillow/blob/main/colab/(%EC%B0%90)FinalCode_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Selenium
# 우리 input값 받아오기
# 일치 체크박스
# EasyOCR
# 우선 지금 전처리 필터로 다른 사진 적용해보기
# 이미지 전처리 필터 찾기&적용 


# -----------------------------------대충 전처리까지 완료

# 해야 할 일
# Selenium
# 전체 값: 0 ==> 사진 재촬영 요구
# 전체 값: 1 ==> 첫번째 링크 열어서 텍스트 보여주기
# 전체 값: n ==> 약학정보원 정보 제공 (--> 이렇게 할 건지, 결과 하나 나오거나 맨 위의 결과가 정답인 알약으로 추려서 테스트 할 건지 고민해보기!)


# 딥러닝 기반 알약 자동 분류 및 정보 제공 시스템
> **Digital Image Processing 설계 프로젝트**<br>
> 2조 <br>
> 1772029 윤소진<br>
> 1772042 정유진<br>
> 1772146 백지원<br>



프로그램 실행을 위한 준비 단계

In [2]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15069, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15069 (delta 1), reused 3 (delta 0), pack-reused 15056
Receiving objects: 100% (15069/15069), 13.44 MiB | 13.40 MiB/s, done.
Resolving deltas: 100% (10235/10235), done.


In [3]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [4]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [5]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

In [6]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [7]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/content
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g6cOUsj5uLk0u02S4VcddYHv9xmun1BrQ5UFaGnIvNHo8bD0akQHfU
Mounted at /content/gdrive


In [8]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'강의 필기 자료 더미'			  'DIP 프로젝트_제안서.gdoc'
 임베디드				   ECE222_assignments_1.pdf
 집적회로설계				  'ECE222_Notes_f19 .pdf'
'1772042랑 공유'			   final_f17_sol.pdf
'중간발표 피피티 - 막대 그래프 1.gsheet'   final-project
'backup from jungyoujin0527'		   GCE_YOUJINJUNG.mp4
'Colab Notebooks'			   MyDrive
 Compal.zip				   resume
'Computer Algorithm.zip'		   text.txt
 dataset				  'Untitled Diagram.png'
'Data Structures.zip'			   yolov4
 DIP					  'Zoom 회의 2021-05-17 15-30-54.mp4'
 DIP프로젝트				  'Zoom 회의 2021-05-17 16-28-34.mp4'


In [9]:
#!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/yolov4/testImg/testImg3.jpg -thresh 0.3#

# Image Segmentation
**yolov4**


In [10]:
%cd /content/
!git clone https://github.com/jung-youjin/yolov4-custom-functions.git

/content
Cloning into 'yolov4-custom-functions'...
remote: Enumerating objects: 241, done.
remote: Total 241 (delta 0), reused 0 (delta 0), pack-reused 241
Receiving objects: 100% (241/241), 62.01 MiB | 24.95 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [11]:
%cd yolov4-custom-functions/

/content/yolov4-custom-functions


In [12]:
# TensorFlow GPU
!pip install -r requirements-gpu.txt

     |████████████████████████████████| 320.4MB 49kB/s 
     |████████████████████████████████| 28.7MB 111kB/s 
     |████████████████████████████████| 460kB 35.8MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 2.9MB 45.5MB/s 
     |████████████████████████████████| 3.0MB 30.6MB/s 
  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13953 sha256=f3cffd5da2cd711e86d640ef7e9150c6c64dd559d595087ebf0256c8210becd7
  Stored in directory: /root/.cache/pip/wheels/81/20/7e/1dd0daad1575d5260916bb1e9781246430647adaef4b3ca3b3
Successfully built pytesseract
ERROR: tensorflow 2.5.0 has requirement gast==0.4.0, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement tensorbo

In [ ]:
%cd /content

In [ ]:
# weight를 /yolov4-custom-functions/data 에 불러오기
!cp /mydrive/yolov4/backup/yolov4-obj_last.weights ./yolov4-custom-functions/data

In [ ]:
# crop할 더미 이미지 data folder에 불러오기
!cp /mydrive/yolov4/testImg/testImg555.JPG ./yolov4-custom-functions/data/images

In [ ]:
%cd /content/yolov4-custom-functions/data/classes
!rm coco.names
%cd /content

In [ ]:
# yolov4 label - pill로 변경
!cp /mydrive/yolov4/obj.names /content/yolov4-custom-functions/data/classes/coco.names
%cd /content/yolov4-custom-functions

## 사전에 미리 학습된 yolov4 weight 불러오기

In [18]:
# coco.names - pill
!python save_model.py --weights ./data/yolov4-obj_last.weights --output ./checkpoints/yolov4-416 --input_size 416 --model yolov4 

2021-06-04 14:48:58.199492: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-06-04 14:48:59.604063: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-06-04 14:48:59.666937: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-06-04 14:48:59.667850: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2021-06-04 14:48:59.667925: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-06-04 14:48:59.880093: I tensorflow/stream_executor/platform/defa

필요한 패키지 설치

In [ ]:
# 필요한 패키지 설치
!pip install easyocr --no-deps # Colab already has all dependencies
!pip install python-bidi
import easyocr
reader = easyocr.Reader(['en'])
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!pip install requests
!pip install beautifulsoup4
!apt install chromium-chromedriver

In [ ]:
# 재실행 경우
%cd /content/yolov4-custom-functions/
!rm -rf ./data/images/test

인식 원하는 이미지 파일 Upload

In [ ]:
# 직접 찍은 알약으로 image upload
!mkdir ./data/images/test
%cd ./data/images/test
from google.colab import files

uploaded = files.upload()
filename = ""
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  filename = fn
print(filename)

# 아무파일 업로드하면 알아서 파일명 변경해주고 알아서 처리함...  
!mv $filename testImg.jpg

In [ ]:
# testImg.jpg crop 
%cd /content/yolov4-custom-functions/
!python detect.py --weights ./checkpoints/yolov4-416 --size 416 --model yolov4 --images ./data/images/test/testImg.jpg --crop

## Crop & Save된 이미지 확인하기

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

path = '/content/yolov4-custom-functions/detections/crop/testImg'
file_list = os.listdir('/content/yolov4-custom-functions/detections/crop/testImg/')
print(len(file_list), file_list)

for i in range(1,len(file_list)+1):
  img = cv2.imread('/content/yolov4-custom-functions/detections/crop/testImg/pill_%s.png' %i, cv2.IMREAD_COLOR) #여기 경로 바꾸기
  imgFile = 'pill_' + str(i) + '.png'

  plotNum = 121
  plt.subplot(plotNum),plt.imshow(img),plt.title('Cropped pill (%s)' %imgFile)
  plt.show()
  plotNum = plotNum + 1

*test data 수집용*

- 테스트 성공하면 그 이미지파일 직접 **google drive** `yolov4/testImg/Test` 에 업로드 부탁합니다
- 테스트 실패하면 그 이미지파일 직접 **google drive**`yolov4/testImg/안되는 약`에 업로드부탁합니다

# Image Preprocessing

## Image Enhancement Filtering

### 영상 Gray scale 변환


### Contrast Limited Adaptive Histogram Equalization 기법

In [ ]:
%cd /content/yolov4-custom-functions/
!mkdir /content/opencv-testImg
!cp -r /content/yolov4-custom-functions/detections/crop/testImg/*.png /content/opencv-testImg

import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

path = '/content/opencv-testImg'
file_list = os.listdir('/content/opencv-testImg/')
print(len(file_list), file_list)

for i in range(1,len(file_list)+1):
  img = cv2.imread('/content/opencv-testImg/pill_%s.png' %i, cv2.IMREAD_GRAYSCALE) #여기 경로 바꾸기
  img_enhanced = cv2.equalizeHist(img)
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  cl1=clahe.apply(img_enhanced)
  #blur5 = cv2.bilateralFilter(img_enhanced, 5,75,75)
  #kernel_sharpen_3 = np.array([[-1,-1,-1,-1,-1],[-1,2,2,2,-1],[-1,2,8,2,-1],[-1,2,2,2,-1],[-1,-1,-1,-1,-1]])/8.0
  #output_1 = cv2.filter2D(cl1,-1,kernel_sharpen_3)
  plt.subplot(121),plt.imshow(img),plt.title('Original')
  plt.subplot(122),plt.imshow(cl1, cmap='gray'),plt.title('Gray + HistEQ + ')
  plt.show()

  save = plt.imsave('pill_%s_histeq.png' %i, cl1, cmap='gray')

!mkdir /content/ocr
!cp -r /content/yolov4-custom-functions/*.png /content/ocr

# Optical Character Recognition

**EasyOCR**

### 전처리된 이미지로부터 문자 인식

In [ ]:
# Doing OCR. Get bounding boxes.
import os
from google.colab.patches import cv2_imshow

import matplotlib.image as mpimg
from PIL import Image

path = '/content/ocr'
file_list = os.listdir('/content/ocr')
print(len(file_list), file_list)

f = open("/content/input.txt", 'w')
f = open("/content/input.txt", 'a')
for i in range(1,len(file_list)+1):
  print(i)
  filename = "pill_%s_histeq.png" % i
  bounds = reader.readtext(filename)
  for j in range(0,len(bounds)):
    data = bounds[j][1]
    data1 = data.replace(" ","")
    f.write(data1)
    print
  f.write("\n")
f.close()



---



---



---



# Web Crawling




**Selenium**

## 약학정보원을 통해 알약 정보 제공

In [ ]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import requests
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

url = 'https://www.health.kr/searchIdentity/search.asp'
# kword = input('검색어를 입력하세요 : ')
# base_url = url + quote_plus(kword)
# base_url
# search = input('input: ')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver.get(url)
browser = driver.get(url)

response = requests.get(url)
soup = bs(response.text, "html.parser")
searches = soup.select('a')

box = driver.find_element_by_xpath('//*[@id="drug_print_front"]')

txtfile = open("/content/input.txt", 'r')
lines = txtfile.readlines()
inc = 1
print("\n-------------------- 알약 자동 인식 결과 정보 제공 ---------------------------\n")

for line in lines:
  driver.get(url)
  response = requests.get(url)
  soup = bs(response.text, "html.parser")
  searches = soup.select('a')
  box = driver.find_element_by_xpath('//*[@id="drug_print_front"]')
  print("\n >> ", line)
  if(line == " " or line == "" or line == "\n"):
    print("알약 내 문자를 인식하지 못했습니다\n")
    print("Failed to detect text on the identified pill\n")

  img = cv2.imread('/content/yolov4-custom-functions/detections/crop/testImg/pill_%s.png' %inc, cv2.IMREAD_COLOR) #여기 경로 바꾸기
  imgFile = 'pill_' + str(inc) + '.png'
 
  plotNum = 121
  plt.subplot(plotNum),plt.imshow(img),plt.title('Identified pill: %s' %line)
  plt.show()
  plotNum = plotNum + 1
  inc = inc + 1


  name = line.replace("\n","")
  box = box.send_keys(name) #box = box.send_keys("tylenol")
  checkbox = driver.find_element_by_xpath('//*[@id="frm"]/fieldset/table/tbody/tr[1]/td[1]/ul/li[1]/div/label')
  checkbox.click()
  searchButton = driver.find_element_by_xpath('//*[@id="btn_idfysearch"]').click() #엔터
  
  driver.implicitly_wait(15) # wait for loading 10seconds

  searchNum = driver.find_element_by_xpath('//*[@id="idfy_total_cnt_view"]')
  driver.implicitly_wait(5) # wait for loading 10seconds
  # print('search 결과값 갯수: ' + searchNum.text)
  searchNumText = searchNum.text

  if (searchNumText == "0" or searchNumText == "24306"):
    print("\n-------- 검색된 결과값이 없습니다 --------------\n")
    print("\n***************************************")
    print("\n********* 다시 촬영해 주세요 **********\n")
    print("***************************************\n")
    print("\n----------------------------------------------------------\n")

  else:
    result = driver.find_element_by_xpath('/html/body/div[1]/section/section/section/article[2]/article/table[1]/tbody/tr[3]/td[2]')
    pillName = line.rstrip("\n")
    print("\n\n-------- 검색된 %s의 약학정보원 결과값 ---------------------------------\n\n" %pillName)
    print(result.text) #엔터
    page = driver.find_element_by_xpath('/html/body/div[1]/section/section/section/article[2]/article/table[1]/tbody/tr[3]/td[7]').click()
    detail = driver.find_element_by_xpath('/html/body/div[1]/section/section/section/article[1]/div/a[1]').click()
    table = driver.find_element_by_xpath('/html/body/div[2]/section/section/section/article[1]/table')
    print(table.text)
    print("\n----------------------------------------------------------\n")
    # try:
    #     WebDriverWait(browser, 3).until(EC.alert_is_present(),
    #                                 'Timed out waiting for PA creation ' +
    #                                 'confirmation popup to appear.')

    #     alert = browser.switch_to.alert
    #     alert.accept()
    #     print("alert accepted")
    # except TimeoutException:
    #     print("no alert")
    #     print(table.text)